In [55]:
from data_prep import DataProcessor, Classement
data_path = '/Users/camille/repo/Hetic/projet_gouv/scraping/Data/google_reviews.csv'

columns = ['date', 'rate', 'review_text', 'object_address']

processor = DataProcessor(data_path)
processor.process_data(columns)
data = processor.data
#Load data2
location_path = '/Users/camille/repo/Hetic/projet_gouv/scraping/Data/location.csv'
# Vous pouvez accéder aux données transformées via l'attribut `data` de l'objet `proc ssor`


classment = Classement(data_path, location_path)
classment.load_data()
classment.load_location_data()
average_ratings_data = classment.get_average_ratings()



AttributeError: 'Classement' object has no attribute 'get_average_ratings'

In [50]:
classment = Classement(data_path, location_path)
classment.load_data()
classment.load_location_data()
average_ratings_data = classment.get_average_ratings()


AttributeError: 'Classement' object has no attribute 'get_average_ratings'

In [6]:
from geopy.geocoders import Nominatim
import pandas as pd
data = pd.read_csv('/Users/camille/repo/Hetic/projet_gouv/scraping/all_data/google_reviews_RGPD.csv')

geolocator = Nominatim(user_agent="my_app")  # Remplacez "my_app" par le nom de votre application
addresses = data['object_address'].unique()

for address in addresses:
    location = geolocator.geocode(address)
    if location is not None:
        latitude = location.latitude
        longitude = location.longitude
    else:
        print(f"Impossible de trouver les coordonnées pour l'adresse: {address}")


Impossible de trouver les coordonnées pour l'adresse: 29 Av. du Général de Gaulle, 52200 Langres
Impossible de trouver les coordonnées pour l'adresse: 47 Rue du Huit Mai, 63500 Issoire
Impossible de trouver les coordonnées pour l'adresse: 1 Terr. Front du Médoc, 33000 Bordeaux
Impossible de trouver les coordonnées pour l'adresse: 1 Av. Maréchal-Juin, 38800 Le Pont-de-Claix
Impossible de trouver les coordonnées pour l'adresse: 13 Bd Edgar Quinet, 79200 Parthenay
Impossible de trouver les coordonnées pour l'adresse: 128 Bd Président Wilson, 73100 Aix-les-Bains
Impossible de trouver les coordonnées pour l'adresse: 4 Rue René Buthaud, 33100 Bordeaux
Impossible de trouver les coordonnées pour l'adresse: 1 Rue du Bois des Fossés, 28100 Dreux
Impossible de trouver les coordonnées pour l'adresse: 51 Av. Général de Gaulle, 03100 Montluçon
Impossible de trouver les coordonnées pour l'adresse: 83 Av. de la Cr de France, 91260 Juvisy-sur-Orge
Impossible de trouver les coordonnées pour l'adresse: 2

KeyboardInterrupt: 

In [7]:
from geopy.geocoders import Nominatim
import pandas as pd

data = pd.read_csv('/Users/camille/repo/Hetic/projet_gouv/scraping/all_data/google_reviews_RGPD.csv')

geolocator = Nominatim(user_agent="my_app")  # Remplacez "my_app" par le nom de votre application
addresses = data['object_address'].unique()

count = 0
total = 0

for address in addresses:
    location = geolocator.geocode(address)
    if location is not None:
        count += 1
    total += 1

pourcentage_coordonnees = (count / total) * 100

print(f"Pourcentage de lieux avec coordonnées : {pourcentage_coordonnees}%")


Pourcentage de lieux avec coordonnées : 84.640522875817%


In [8]:
from geopy.geocoders import Nominatim
import pandas as pd

data = pd.read_csv('/Users/camille/repo/Hetic/projet_gouv/scraping/all_data/google_reviews_RGPD.csv')

geolocator = Nominatim(user_agent="my_app")  # Remplacez "my_app" par le nom de votre application
addresses = data['object_address'].unique()

coordinates = []

# Dictionnaire pour mettre en cache les coordonnées des villes déjà recherchées
cache = {}

for address in addresses:
    city = address.split(",")[-1].strip()  # Extraire le nom de la ville à partir de l'adresse
    
    # Vérifier si les coordonnées de la ville sont déjà en cache
    if city in cache:
        latitude, longitude = cache[city]
    else:
        location = geolocator.geocode(city)
        if location is not None:
            latitude = location.latitude
            longitude = location.longitude
            cache[city] = (latitude, longitude)  # Mettre les coordonnées en cache pour une utilisation ultérieure
        else:
            print(f"Impossible de trouver les coordonnées pour la ville: {city}")
            continue

    coordinates.append((address, latitude, longitude))

missing_coordinates = len(addresses) - len(coordinates)
pourcentage_coordonnees = (len(coordinates) / len(addresses)) * 100

print(f"Pourcentage de lieux avec coordonnées : {pourcentage_coordonnees}%")
print(f"Nombre de lieux sans coordonnées : {missing_coordinates}")

# Créer un DataFrame avec les coordonnées
df_coordinates = pd.DataFrame(coordinates, columns=['address', 'latitude', 'longitude'])
data.rename(columns={'object_address': 'address'}, inplace=True)
# Fusionner les coordonnées avec le DataFrame d'origine
data = pd.merge(data, df_coordinates, on='address', how='left')

# Enregistrer les données avec les coordonnées dans un fichier CSV
data.to_csv('data_with_coordinates.csv', index=False)


Pourcentage de lieux avec coordonnées : 100.0%
Nombre de lieux sans coordonnées : 0


KeyError: 'address'

In [9]:
# Créer un DataFrame avec les coordonnées
df_coordinates = pd.DataFrame(coordinates, columns=['address', 'latitude', 'longitude'])
data.rename(columns={'object_address': 'address'}, inplace=True)
# Fusionner les coordonnées avec le DataFrame d'origine
data = pd.merge(data, df_coordinates, on='address', how='left')

# Enregistrer les données avec les coordonnées dans un fichier CSV
data.to_csv('data_with_coordinates.csv', index=False)

In [21]:
import plotly.express as px
df_grouped = data.groupby('address').agg({'rate': [('average_rate', 'mean'), ('review_count', 'count')], 'latitude': 'first', 'longitude': 'first'}).reset_index()
df_grouped.columns = ['address', 'average_rate', 'review_count', 'latitude', 'longitude']

df_grouped['average_rate'] = df_grouped['average_rate'].round(1)

map = px.scatter_mapbox(df_grouped, lat=df_grouped.latitude, lon=df_grouped.longitude, hover_name="address", hover_data=["average_rate", "review_count"])
map.show()


In [25]:
import plotly.graph_objects as go

df_grouped = data.groupby('address').agg({'rate': [('average_rate', 'mean'), ('review_count', 'count')], 'latitude': 'first', 'longitude': 'first'}).reset_index()
df_grouped.columns = ['address', 'average_rate', 'review_count', 'latitude', 'longitude']

df_grouped['average_rate'] = df_grouped['average_rate'].round(1)

fig = go.Figure(go.Scattermapbox(
    lat=df_grouped['latitude'],
    lon=df_grouped['longitude'],
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=10,
        color='rgb(0, 0, 255)',
        opacity=0.7
    ),
    text=df_grouped['address'],
    hovertemplate="<b>%{text}</b><br><br>" +
                  "Average Rate: %{marker.size:.1f}<br>" +
                  "Review Count: %{marker.opacity}<br>" +
                  "<extra></extra>"
))

fig.update_layout(
    mapbox_style='open-street-map',
    mapbox_zoom=5,
    mapbox_center={'lat': df_grouped['latitude'].mean(), 'lon': df_grouped['longitude'].mean()},
)

fig.show()


df_grouped